In [1]:
import tensorflow as tf
import librosa
import numpy as np

/home/metlab/venv/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [10]:
tf.constant(1.0, dtype=tf.float32) + tf.constant(1.0, dtype=tf.float32)*1j

TypeError: unsupported operand type(s) for *: 'Tensor' and 'complex'

In [16]:
from IPython.display import Audio
from IPython.core.display import display

In [17]:
y, fs = librosa.load('../superchip/superchip/heaven.wav')

In [18]:
y = y[:fs*10]

In [5]:
with tf.Session() as sess:
    x_time = tf.placeholder(tf.float32, shape=[None, y.shape[0]], name=None)
    x_time_var = tf.Variable(np.random.randn(1,y.shape[0]).astype(np.float32))
    
    x_stft = tf.contrib.signal.stft(x_time, 2048, 512)
    x_stft_var = tf.contrib.signal.stft(x_time_var, 2048, 512)
    
    mse_loss = tf.reduce_sum(tf.abs(x_stft - x_stft_var)**2)
    
    grads = tf.gradients([mse_loss], x_time_var)[0]
    
    init = tf.global_variables_initializer()
    sess.run(init)
    
    grads_eval = sess.run(grads, feed_dict={x_time: y[np.newaxis,:]})
    
    opt = tf.contrib.opt.ScipyOptimizerInterface(mse_loss, var_list=[x_time_var])
    opt.minimize(sess, feed_dict={x_time: y[np.newaxis,:]})
    
    x_evaled = sess.run(x_time_var).squeeze()

INFO:tensorflow:Optimization terminated with:
  Message: b'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH'
  Objective function value: 0.000358
  Number of iterations: 8133
  Number of functions evaluations: 8162


In [6]:
display(Audio(x_evaled, rate=22050))

In [7]:
with tf.Session() as sess:
    x_time = tf.placeholder(tf.float32, shape=[None, y.shape[0]], name=None)
    x_time_var = tf.Variable(np.random.randn(1,y.shape[0]).astype(np.float32))
    
    x_stft = tf.abs(tf.contrib.signal.stft(x_time, 2048, 512))
    x_stft_var = tf.abs(tf.contrib.signal.stft(x_time_var, 2048, 512))
    
    mse_loss = tf.reduce_sum(tf.abs(x_stft - x_stft_var)**2)
    
    grads = tf.gradients([mse_loss], x_time_var)[0]
    
    init = tf.global_variables_initializer()
    sess.run(init)
    
    grads_eval = sess.run(grads, feed_dict={x_time: y[np.newaxis,:]})
    
    opt = tf.contrib.opt.ScipyOptimizerInterface(mse_loss, var_list=[x_time_var])
    opt.minimize(sess, feed_dict={x_time: y[np.newaxis,:]})
    
    x_evaled = sess.run(x_time_var).squeeze()

INFO:tensorflow:Optimization terminated with:
  Message: b'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH'
  Objective function value: 1302.550537
  Number of iterations: 3371
  Number of functions evaluations: 3542


In [8]:
display(Audio(x_evaled, rate=22050))

In [30]:
def normalize(y):
    return y/np.abs(y).max()

In [31]:
y = normalize(y)

In [33]:
with tf.Graph().as_default() as g:
    
    config = tf.ConfigProto(allow_soft_placement = True)
    config.gpu_options.allow_growth = True
    device = get_available_devices()[0]
    
    with g.device(device):
        x_time = tf.placeholder(tf.float32, shape=[None, y.shape[0]], name=None)
        x_time_var = tf.Variable(np.random.randn(1,y.shape[0]).astype(np.float32))

        x_stft = tf.contrib.signal.stft(x_time, 2048, 512)
        x_stft_var = tf.contrib.signal.stft(x_time_var, 2048, 512)

        mse_loss = tf.reduce_sum(tf.abs(x_stft - x_stft_var)**2)

        grads = tf.gradients([mse_loss], x_time_var)[0]

        with tf.Session(config=config) as sess:
            init = tf.global_variables_initializer()
            sess.run(init)

            grads_eval = sess.run(grads, feed_dict={x_time: y[np.newaxis,:]})

            x_evaled = sess.run(x_time_var)
            x_stft_evaled = sess.run(x_stft, feed_dict={x_time: y[np.newaxis,:]})
            print(x_stft_evaled.shape)
            print(x_stft_evaled.min(), x_stft_evaled.max())

Exception ignored in: <bound method BaseSession.__del__ of <tensorflow.python.client.session.Session object at 0x7f3acc055470>>
Traceback (most recent call last):
  File "/home/metlab/venv/lib/python3.5/site-packages/tensorflow/python/client/session.py", line 704, in __del__
    if self._session is not None:
AttributeError: 'Session' object has no attribute '_session'


(1, 427, 1025)
(-164.56493-46.23856j) (165.37589+15.176516j)


In [7]:
from tensorflow.python.client import device_lib

In [9]:
def get_available_devices():
    local_device_protos = device_lib.list_local_devices()
    devices = [x.name for x in local_device_protos if x.device_type == 'GPU']
    if devices == []:
        return ['/cpu:0']
    else:
        return devices

In [10]:
get_available_devices()

['/device:GPU:0']